#Features Extraction Using ProtBERT Model

In [ ]:
!pip install transformers==4.28.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 52.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from transformers import BertModel, BertTokenizer
import re
import pandas as pd
import torch
import numpy as np
df1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/AMP-2023/ProtBERT-BFD-Features-ACP740.csv')

In [ ]:
df1

,0,1,2,3,4,5,6,7,8,9,...,101367,101368,101369,101370,101371,101372,101373,101374,101375,label
0,0.093729,-0.102747,0.057787,0.033196,-0.105783,-0.123355,0.003799,-0.015746,-0.012691,0.024303,...,-0.011490,0.074752,-0.006655,0.160421,-0.028422,0.146628,-0.041246,0.028337,0.130006,1
1,0.081728,0.005799,0.122330,0.051133,-0.086127,-0.083545,-0.005902,0.008309,0.056063,0.072475,...,0.063172,0.000027,0.037047,0.227829,-0.063908,0.013715,-0.018802,0.039775,0.047436,1
2,0.064492,-0.009263,0.125742,0.072312,-0.095630,-0.090610,-0.004768,0.030741,0.015981,0.028946,...,0.033422,0.047389,0.024299,0.153164,0.029398,0.020481,-0.021634,0.014946,0.047036,1
3,0.065506,0.004920,0.135514,0.047689,-0.082039,-0.052439,0.002737,0.035399,0.037714,0.032775,...,0.029946,0.048047,0.011084,0.137034,0.029809,0.027347,-0.016527,0.021118,0.051146,1
4,0.069772,-0.050748,0.133196,0.058087,-0.116573,-0.086057,0.011390,0.001331,0.019568,0.019814,...,0.054273,0.041598,-0.004001,0.177913,0.003600,0.022875,-0.009635,0.045603,0.046496,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
735,0.050200,-0.091386,0.080422,0.029051,-0.004771,-0.133247,-0.073690,0.002180,0.123816,-0.036413,...,-0.061748,-0.099389,-0.009764,0.078906,0.044151,0.044070,-0.027789,0.030316,0.046510,0
736,0.089163,-0.089297,0.141592,0.047706,-0.088163,-0.103521,-0.082961,0.004666,0.037096,0.015094,...,0.038667,0.063180,-0.005966,0.155855,-0.001411,0.020944,-0.034143,0.057790,0.018514,0
737,0.064728,0.020542,0.087482,0.044555,0.021291,-0.060406,-0.021713,0.021270,-0.025757,-0.061631,...,-0.026440,0.028017,0.036465,0.177421,-0.053503,-0.017123,-0.020355,-0.031031,0.100115,0
738,0.052701,0.026202,0.149011,0.053000,-0.030129,-0.141054,0.027376,0.035426,0.019657,0.000608,...,0.015040,0.056802,-0.015052,-0.034291,0.007914,0.012234,-0.064678,-0.046626,-0.004542,0


In [ ]:
# Upload the Dataset in FASTA
peptides = []
labels = []
with open("/content/drive/MyDrive/Colab Notebooks/ACP/Datasets/acp740.txt", "r") as file:
    for line in file:
        line = line.strip()
        if len(line) == 0:
            continue
        if line[0] == ">":
            labels.append(int(line[-1]))
        else:
            peptides.append(line)

In [ ]:
# Convert to a DataFrame
data = {'aa_seq': peptides, 'Prediction': labels}
df1 = pd.DataFrame(data)
print(df1)

                                aa_seq  Prediction
0    GLWSKIKEVGKEAAKAAAKAAGKAALGAVSEAV           1
1                        GLFDIIKKIAESI           1
2                     GLLDIVKKVVGAFGSL           1
3                     GLFDIVKKVVGALGSL           1
4                     GLFDIVKKVVGTLAGL           1
..                                 ...         ...
735                      VRRFPWWWPFLRR           0
736      RRRRRRRRGEDIIRNIARHLAQVGDSMDR           0
737               VYINKLTPPCGTMYYACEAV           0
738          ACDCRGDCFCGGGGIVRRADRAAVP           0
739                        GSSSGRGDSPA           0

[740 rows x 2 columns]


In [ ]:
#df = df1
#df.head()


In [ ]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False)
model = BertModel.from_pretrained("Rostlab/prot_bert")
sequence_examples = df1['aa_seq']
sequence_examples = [" ".join(list(re.sub(r"[UZOB]", "X", sequence))) for sequence in sequence_examples]



(…)lab/prot_bert_bfd/resolve/main/vocab.txt:   0%|          | 0.00/81.0 [00:00<?, ?B/s]

(…)bfd/resolve/main/special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

(…)t_bfd/resolve/main/tokenizer_config.json:   0%|          | 0.00/86.0 [00:00<?, ?B/s]

(…)b/prot_bert_bfd/resolve/main/config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.68G [00:00<?, ?B/s]

Some weights of the model checkpoint at Rostlab/prot_bert_bfd were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
ids = tokenizer.batch_encode_plus(sequence_examples, add_special_tokens=True, padding="longest")
input_ids = torch.tensor(ids['input_ids']).to(device)
attention_mask = torch.tensor(ids['attention_mask']).to(device)


In [ ]:
with torch.no_grad():
    output = model(input_ids=input_ids, attention_mask=attention_mask)

In [ ]:
output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[ 0.0937, -0.1027,  0.0578,  ..., -0.0886, -0.1661, -0.0263],
         [ 0.1185, -0.0254,  0.0751,  ...,  0.0874, -0.0876,  0.0842],
         [ 0.1299, -0.0494,  0.0863,  ..., -0.1761,  0.0770,  0.0208],
         ...,
         [-0.1069, -0.0714, -0.0536,  ...,  0.0348, -0.0998,  0.0355],
         [-0.1707, -0.0794, -0.0673,  ...,  0.0053,  0.0179,  0.1382],
         [-0.1059, -0.0826, -0.0465,  ..., -0.0412,  0.0283,  0.1300]],

        [[ 0.0817,  0.0058,  0.1223,  ..., -0.0911, -0.1160, -0.0369],
         [ 0.0666, -0.0252,  0.0193,  ...,  0.0190, -0.1180,  0.0380],
         [ 0.0937,  0.0097,  0.0411,  ..., -0.1580, -0.0018,  0.0323],
         ...,
         [-0.0284, -0.0341, -0.0156,  ...,  0.0091, -0.0355,  0.0289],
         [ 0.1098, -0.0359,  0.0171,  ..., -0.1327,  0.0327,  0.0201],
         [ 0.0418, -0.0183,  0.0076,  ..., -0.0188,  0.0398,  0.0474]],

        [[ 0.0645, -0.0093,  0.1257,  ..., -0.0918, -

In [ ]:
# Forward pass through the model
#with torch.no_grad():
#    output = model(**encoded_input)

# Extract the features
features = output.last_hidden_state.squeeze().detach().numpy()


In [ ]:
features

array([[[ 0.09372935, -0.10274705,  0.05778697, ..., -0.08862302,
         -0.16610369, -0.02632569],
        [ 0.11846219, -0.02537626,  0.07512266, ...,  0.08744717,
         -0.08755492,  0.08422205],
        [ 0.12992094, -0.04937679,  0.08633039, ..., -0.17605017,
          0.07696003,  0.0208391 ],
        ...,
        [-0.10690386, -0.07144731, -0.05363042, ...,  0.03481168,
         -0.09983663,  0.03553772],
        [-0.17066605, -0.07943527, -0.06731749, ...,  0.00533216,
          0.01793376,  0.13823894],
        [-0.10590126, -0.08263171, -0.04652411, ..., -0.04124611,
          0.02833723,  0.13000646]],

       [[ 0.08172756,  0.00579855,  0.12232998, ..., -0.09113852,
         -0.11599429, -0.03685885],
        [ 0.06658203, -0.02517762,  0.01925263, ...,  0.01896005,
         -0.11795856,  0.03798718],
        [ 0.09365772,  0.00967836,  0.0411001 , ..., -0.15801227,
         -0.00179697,  0.03226055],
        ...,
        [-0.02844387, -0.03412609, -0.01557971, ...,  

In [ ]:
output.keys()

odict_keys(['last_hidden_state', 'pooler_output'])

In [ ]:
output['pooler_output']

tensor([[-0.3232, -0.2725,  0.3200,  ..., -0.3197, -0.3264,  0.3209],
        [-0.3279, -0.2767,  0.3247,  ..., -0.3244, -0.3311,  0.3256],
        [-0.3231, -0.2724,  0.3200,  ..., -0.3197, -0.3263,  0.3209],
        ...,
        [-0.3240, -0.2732,  0.3208,  ..., -0.3206, -0.3272,  0.3218],
        [-0.3298, -0.2785,  0.3266,  ..., -0.3264, -0.3331,  0.3276],
        [-0.3254, -0.2744,  0.3222,  ..., -0.3219, -0.3286,  0.3231]])

In [ ]:
output['last_hidden_state']

tensor([[[ 0.0937, -0.1027,  0.0578,  ..., -0.0886, -0.1661, -0.0263],
         [ 0.1185, -0.0254,  0.0751,  ...,  0.0874, -0.0876,  0.0842],
         [ 0.1299, -0.0494,  0.0863,  ..., -0.1761,  0.0770,  0.0208],
         ...,
         [-0.1069, -0.0714, -0.0536,  ...,  0.0348, -0.0998,  0.0355],
         [-0.1707, -0.0794, -0.0673,  ...,  0.0053,  0.0179,  0.1382],
         [-0.1059, -0.0826, -0.0465,  ..., -0.0412,  0.0283,  0.1300]],

        [[ 0.0817,  0.0058,  0.1223,  ..., -0.0911, -0.1160, -0.0369],
         [ 0.0666, -0.0252,  0.0193,  ...,  0.0190, -0.1180,  0.0380],
         [ 0.0937,  0.0097,  0.0411,  ..., -0.1580, -0.0018,  0.0323],
         ...,
         [-0.0284, -0.0341, -0.0156,  ...,  0.0091, -0.0355,  0.0289],
         [ 0.1098, -0.0359,  0.0171,  ..., -0.1327,  0.0327,  0.0201],
         [ 0.0418, -0.0183,  0.0076,  ..., -0.0188,  0.0398,  0.0474]],

        [[ 0.0645, -0.0093,  0.1257,  ..., -0.0918, -0.1132, -0.0522],
         [ 0.0750, -0.0361,  0.0027,  ...,  0

In [ ]:
# Reshape the features to 2D vector space
num_sequences, max_sequence_length, hidden_size = features.shape
features_2d = features.reshape(num_sequences, max_sequence_length * hidden_size)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD

In [ ]:
# Apply SVD for dimensionality reduction
svd = TruncatedSVD(n_components=1024)  # Adjust the number of components as desired
reduced_features = svd.fit_transform(features_2d)

In [ ]:
reduced_features

array([[ 8.7964268e+00,  1.7042130e+01, -8.0169325e+00, ...,
        -1.2896798e-02,  1.0355735e-02, -1.9216347e-02],
       [ 1.6806541e+01,  9.1772270e+00, -1.0620022e+01, ...,
        -2.4606697e-02, -2.1756466e-01, -6.5632403e-02],
       [ 1.1581911e+01,  1.1281575e+01, -1.1806218e+01, ...,
        -2.3192912e-01, -2.7987256e+00,  7.8503810e-02],
       ...,
       [ 1.8933323e+01, -7.6897535e+00, -2.5088396e+00, ...,
         3.5359822e-02,  4.7803070e-02, -2.9797399e-02],
       [ 1.5536141e+01, -1.3165790e+01,  4.3995194e+00, ...,
         2.0791488e-02,  1.6113822e-03,  3.9828313e-03],
       [ 1.4603464e+01, -9.3074007e+00, -2.1896796e-01, ...,
        -2.3102596e-02,  1.2135264e-03, -6.1639091e-03]], dtype=float32)

In [ ]:

shape = np.shape(reduced_features)
shape

(740, 740)

In [ ]:
df = pd.DataFrame(reduced_features)
df['label'] = df1['Prediction']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 740 entries, 0 to 739
Columns: 741 entries, 0 to label
dtypes: float32(740), int64(1)
memory usage: 2.1 MB


In [ ]:
print(df.head(5))

           0          1          2          3         4         5         6  \
0   8.796427  17.042130  -8.016932  -8.009449  8.181319 -0.616691  6.639025   
1  16.806541   9.177227 -10.620022  -4.533488 -5.457982 -0.329739  4.072353   
2  11.581911  11.281575 -11.806218 -10.818443 -0.037591  1.806796  1.255808   
3  14.982101   9.205626 -11.908853 -11.051626 -0.512576  0.722628  1.187620   
4  15.722834   9.955873 -11.184158 -10.404865 -0.463884  0.818342  2.689561   

          7         8         9  ...       731       732       733       734  \
0 -3.971672 -4.420681 -2.688262  ... -0.024455  0.022157  0.005620 -0.020652   
1 -4.560667  2.374066  3.315322  ...  0.050041  0.095698 -0.004165  0.214693   
2 -1.342968  3.181098  3.053202  ... -0.234318 -0.276775  0.120002 -0.131404   
3 -1.405514  2.984911  2.857614  ...  0.248547  0.143131  0.140986  0.079913   
4 -2.964561  1.386514  1.724561  ... -0.217798  0.189535 -0.266715  0.025596   

        735       736       737       738   

In [ ]:

# Save the DataFrame to a CSV file
df.to_csv('SVD_extracted_features_ProtBERT-BFD_ACP740.csv', index=False)